In [1]:
"""
Edge AI Recyclable Items Classification
TensorFlow Lite Model for Real-time Recognition
"""

import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import layers, models
import time

print("TensorFlow version:", tf.__version__)

# Load and preprocess dataset (using CIFAR-10 for demonstration)
def load_data():
    """Load and preprocess training data"""
    (train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.cifar10.load_data()

    # Simulate recyclable classes: paper, plastic, glass, metal, non-recyclable
    class_names = ['paper', 'plastic', 'glass', 'metal', 'non-recyclable',
                   'cardboard', 'electronics', 'organic', 'textiles', 'other']

    # Normalize pixel values
    train_images = train_images.astype('float32') / 255.0
    test_images = test_images.astype('float32') / 255.0

    return (train_images, train_labels), (test_images, test_labels), class_names

def create_lightweight_model(input_shape=(32, 32, 3), num_classes=10):
    """Create a lightweight CNN model optimized for Edge devices"""

    model = models.Sequential([
        # First convolutional block
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),

        # Second convolutional block
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),

        # Third convolutional block
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.BatchNormalization(),

        # Classification head
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dropout(0.3),
        layers.Dense(num_classes, activation='softmax')
    ])

    return model

def train_model(model, train_images, train_labels, test_images, test_labels):
    """Train the model with optimization for edge deployment"""

    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    history = model.fit(train_images, train_labels,
                        epochs=15,
                        validation_data=(test_images, test_labels),
                        batch_size=64,
                        verbose=1)

    return history, model

def convert_to_tflite(model):
    """Convert the model to TensorFlow Lite format"""

    # Create converter
    converter = tf.lite.TFLiteConverter.from_keras_model(model)

    # Optimize for size and latency
    converter.optimizations = [tf.lite.Optimize.DEFAULT]

    # Optional: Use float16 quantization for smaller size
    converter.target_spec.supported_types = [tf.float16]

    # Convert model
    tflite_model = converter.convert()

    # Save the model
    with open('recyclable_classifier.tflite', 'wb') as f:
        f.write(tflite_model)

    return tflite_model

def evaluate_tflite_model(tflite_model, test_images, test_labels):
    """Evaluate the TensorFlow Lite model"""

    # Initialize interpreter
    interpreter = tf.lite.Interpreter(model_content=tflite_model)
    interpreter.allocate_tensors()

    # Get input and output tensors
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    # Run predictions
    predictions = []
    inference_times = []

    for i in range(len(test_images[:1000])):  # Test on subset
        # Preprocess input
        input_data = test_images[i:i+1].astype(np.float32)

        # Set tensor
        interpreter.set_tensor(input_details[0]['index'], input_data)

        # Measure inference time
        start_time = time.time()
        interpreter.invoke()
        end_time = time.time()

        inference_times.append((end_time - start_time) * 1000)  # Convert to ms

        # Get prediction
        output_data = interpreter.get_tensor(output_details[0]['index'])
        predictions.append(np.argmax(output_data[0]))

    # Calculate accuracy
    accuracy = np.mean(np.array(predictions) == test_labels[:1000].flatten())
    avg_inference_time = np.mean(inference_times)

    return accuracy, avg_inference_time, predictions

def compare_performance(keras_model, tflite_model, test_images, test_labels):
    """Compare performance between Keras and TFLite models"""

    # Keras model performance
    keras_start = time.time()
    keras_predictions = keras_model.predict(test_images[:1000])
    keras_time = (time.time() - keras_start) * 1000 / 1000  # ms per sample

    keras_accuracy = np.mean(np.argmax(keras_predictions, axis=1) == test_labels[:1000].flatten())

    # TFLite performance
    tflite_accuracy, tflite_time, _ = evaluate_tflite_model(tflite_model, test_images, test_labels)

    print("\n" + "="*50)
    print("PERFORMANCE COMPARISON")
    print("="*50)
    print(f"Keras Model - Accuracy: {keras_accuracy:.4f}, Inference Time: {keras_time:.2f}ms")
    print(f"TFLite Model - Accuracy: {tflite_accuracy:.4f}, Inference Time: {tflite_time:.2f}ms")
    print(f"Size Reduction: {((len(tflite_model) / len(keras_model.get_weights())) * 100):.1f}%")

    return {
        'keras_accuracy': keras_accuracy,
        'keras_inference_time': keras_time,
        'tflite_accuracy': tflite_accuracy,
        'tflite_inference_time': tflite_time,
        'model_size_ratio': len(tflite_model) / len(keras_model.get_weights())
    }

# Main execution
def main():
    print("🚀 Edge AI Recyclable Classification Prototype")
    print("="*50)

    # Load data
    print("1. Loading and preprocessing data...")
    (train_images, train_labels), (test_images, test_labels), class_names = load_data()

    # Create model
    print("2. Creating lightweight CNN model...")
    model = create_lightweight_model()
    print("Model Architecture:")
    model.summary()

    # Train model
    print("3. Training model...")
    history, trained_model = train_model(model, train_images, train_labels, test_images, test_labels)

    # Convert to TFLite
    print("4. Converting to TensorFlow Lite...")
    tflite_model = convert_to_tflite(trained_model)
    print(f"TFLite model size: {len(tflite_model) / 1024:.2f} KB")

    # Evaluate
    print("5. Evaluating models...")
    performance = compare_performance(trained_model, tflite_model, test_images, test_labels)

    # Demonstrate Edge AI benefits
    print("\n" + "="*50)
    print("EDGE AI BENEFITS DEMONSTRATION")
    print("="*50)
    print("✅ Real-time Processing: {:.2f}ms per image".format(performance['tflite_inference_time']))
    print("✅ Offline Capability: No internet required")
    print("✅ Privacy: Image data never leaves the device")
    print("✅ Low Power: Optimized for mobile/edge devices")
    print("✅ Instant Response: Suitable for real-time sorting systems")

    # Save model for deployment
    trained_model.save('recyclable_classifier_keras.h5')
    print("\n✅ Models saved: 'recyclable_classifier_keras.h5' and 'recyclable_classifier.tflite'")

if __name__ == "__main__":
    main()

TensorFlow version: 2.19.0
🚀 Edge AI Recyclable Classification Prototype
1. Loading and preprocessing data...
170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
2. Creating lightweight CNN model...
Model Architecture:


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 30, 30, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 30, 30, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 15, 15, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 13, 13, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 13, 13, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 6, 6, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 4, 4, 64)       │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 4, 4, 64)       │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │        65,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 123,210 (481.29 KB)

 Trainable params: 122,890 (480.04 KB)

 Non-trainable params: 320 (1.25 KB)

3. Training model...
Epoch 1/15
782/782 ━━━━━━━━━━━━━━━━━━━━ 97s 120ms/step - accuracy: 0.3925 - loss: 1.7731 - val_accuracy: 0.5329 - val_loss: 1.3507
Epoch 2/15
782/782 ━━━━━━━━━━━━━━━━━━━━ 91s 116ms/step - accuracy: 0.6082 - loss: 1.1133 - val_accuracy: 0.6408 - val_loss: 1.0135
Epoch 3/15
782/782 ━━━━━━━━━━━━━━━━━━━━ 141s 115ms/step - accuracy: 0.6673 - loss: 0.9473 - val_accuracy: 0.6574 - val_loss: 0.9881
Epoch 4/15
782/782 ━━━━━━━━━━━━━━━━━━━━ 142s 115ms/step - accuracy: 0.7102 - loss: 0.8375 - val_accuracy: 0.5388 - val_loss: 1.4334
Epoch 5/15
782/782 ━━━━━━━━━━━━━━━━━━━━ 91s 117ms/step - accuracy: 0.7328 - loss: 0.7636 - val_accuracy: 0.6887 - val_loss: 0.8914
Epoch 6/15
782/782 ━━━━━━━━━━━━━━━━━━━━ 140s 115ms/step - accuracy: 0.7577 - loss: 0.6985 - val_accuracy: 0.6803 - val_loss: 0.9288
Epoch 7/15
782/782 ━━━━━━━━━━━━━━━━━━━━ 90s 115ms/step - accuracy: 0.7664 - loss: 0.6567 - val_accuracy: 0.6788 - val_loss: 0.9702
Epoch 8/15
782/782 ━━━━━━━━━━━━━━━━━━━━ 93s 119ms/step - ac

/usr/local/lib/python3.12/dist-packages/tensorflow/lite/python/interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)



PERFORMANCE COMPARISON
Keras Model - Accuracy: 0.7570, Inference Time: 1.25ms
TFLite Model - Accuracy: 0.7570, Inference Time: 0.19ms
Size Reduction: 1151309.1%

EDGE AI BENEFITS DEMONSTRATION
✅ Real-time Processing: 0.19ms per image
✅ Offline Capability: No internet required
✅ Privacy: Image data never leaves the device
✅ Low Power: Optimized for mobile/edge devices
✅ Instant Response: Suitable for real-time sorting systems

✅ Models saved: 'recyclable_classifier_keras.h5' and 'recyclable_classifier.tflite'
